# hate-speech-dataset



In [74]:
import pandas as pd
import numpy as np

In [75]:
data_dir = './data/'

In [76]:
df = pd.read_csv(data_dir+'annotations_metadata.csv')

In [77]:
(df.label == 'hate').sum() / len(df) , (df.label == 'hate').sum()

(0.10928362573099415, 1196)

In [78]:
import os

In [94]:
sampled_train_list = [i if ('.txt' in i) else 'marker' for i in os.listdir(data_dir+'sampled_train/')]
sampled_test_list = [i if ('.txt' in i) else 'marker' for i in os.listdir(data_dir+'sampled_test/')]

while 'marker' in sampled_train_list:
    sampled_train_list.remove('marker')
while 'marker' in sampled_test_list:
    sampled_test_list.remove('marker')

len(sampled_train_list)+ len(sampled_test_list)

2392

In [80]:
df['text'] = df['file_id']

In [81]:
def id_file_reader(col):
    if (col['file_id']+'.txt') in sampled_train_list:
        file = os.path.join(data_dir+'sampled_train/',col['file_id']+'.txt')
        with open(file,'r') as f:
            return f.read()
        
    if (col['file_id']+'.txt') in sampled_test_list:
        file = os.path.join(data_dir+'sampled_test/',col['file_id']+'.txt')
        with open(file,'r') as f:
            return f.read()
        
    else: return False

In [82]:
def hate_label(col):
    if col['label'] =='hate':
        return 1
    else:
        return 0

In [83]:
df['comment_text'] = df[['file_id']].apply(id_file_reader, axis = 1)

In [84]:
df_train = df[df.comment_text != False]

In [85]:
df_train['label_int'] = df_train[['label']].apply(hate_label, axis =1 )

/Users/rene/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [86]:
df_train.head(2)

,file_id,user_id,subforum_id,num_contexts,label,text,comment_text,label_int
1,12834217_2,572066,1346,0,noHate,12834217_2,In order to help increase the booklets downloa...,0
3,12834217_4,572066,1346,0,hate,12834217_4,Click below for a FREE download of a colorfull...,1


In [87]:
import re 
rep_numbers=re.compile(r'\d+',re.IGNORECASE)
rep_special_chars=re.compile(r'[^a-z\d ]',re.IGNORECASE)
rep_special_chars= re.compile("[^\w']|_") 

#stemmer = PorterStemmer()
#nltk.download("stopwords", quiet=True)

def review_to_words(review):
    
    text=rep_special_chars.sub(' ', review)
    
    text = rep_numbers.sub('n', text) # substitute all numbers
    
    
    words = text.split() # Split string into words
    words = [w for w in words if len(w) == len(w.encode())][:100]  
    
    return ' '.join(words)

In [88]:
df_train.comment_text

1        In order to help increase the booklets downloa...
3        Click below for a FREE download of a colorfull...
10       She may or may not be a Jew but she 's certain...
20               I think you should write a book as well .
21       I 've always considered teaching as one of the...
22       And the sad thing is the white students at tho...
23       I guess I 'll just stick to homeschooling my k...
31       Their gods often had pale skin and blue eyes b...
34       This whole situation starts to become really b...
42       Greece need a man like Adolf Hitler and now is...
46                         He is a pathetic little chimp .
50       com/watch ? v = kBD6Gm6VoDM Then watch 8 min. ...
59       Try El Paso Texas Soo much mudd invasion that ...
60                          Welcome to the front lines ...
62       Here is a video of the event that is worth wat...
66       Nice to know that are still a few men out ther...
70       I suppose the concept of a male cow and a fema.

In [89]:
df_train['comment_text'] = df_train.comment_text.apply(review_to_words)

/Users/rene/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [90]:
dir_train_json= os.path.join(data_dir+'sampled_train/','train.json')
df_train.to_json(dir_train_json , orient='records', lines=True)

In [91]:
dir_train_csv= os.path.join(data_dir,'train_preprocessed.csv')
df_train.to_csv(dir_train_csv, header=True, index=True)

In [92]:
len(df_train)

2392